<a href="https://colab.research.google.com/github/cammylexi/CS2341-Assignment-3/blob/main/Lab_McPhaul_Llanes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/animal-faces")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/animal-faces


For license plate detection, I recommend these metrics:

Mean Average Precision (mAP): This is the standard evaluation metric for object detection tasks, calculated across different
IoU thresholds.

F1-Score: The harmonic mean of precision and recall, providing a balanced measure of detection performance.

IoU (Intersection over Union): Directly measures how well the predicted bounding boxes match the ground truth.

<br>

Justification: In license plate detection for applications like traffic monitoring or parking management, both false positives and false negatives can have significant impacts. False positives might lead to incorrect vehicle identification or unnecessary processing, while false negatives could result in missed vehicles. The mAP metric provides a comprehensive evaluation across different confidence thresholds, while F1-score gives a balanced view of precision and recall. IoU directly measures localization accuracy, which is crucial for downstream tasks like character recognition.

In [2]:
import os
import cv2
import numpy as np

train_images_path = os.path.join(path, "afhq", "train")
print(f"Train images path: {train_images_path}")

# Define the mapping from class name to one-hot vector
class_names = ['cat', 'dog', 'wild']
class_to_onehot = {
    name: np.eye(len(class_names))[i] for i, name in enumerate(class_names)
}

def load_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    return img

def load_all_images_and_labels(root_dir, target_size=(224, 224)):
    images = []
    labels = []

    for class_name in class_names:
        class_dir = os.path.join(root_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        for file in os.listdir(class_dir):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(class_dir, file)
                img = load_image(image_path, target_size)
                images.append(img)
                labels.append(class_to_onehot[class_name])

    return np.array(images), np.array(labels)

# Example usage
all_images, all_labels = load_all_images_and_labels(train_images_path)
print(f"Loaded {len(all_images)} images and {len(all_labels)} labels.")


Train images path: /kaggle/input/animal-faces/afhq/train
Loaded 14630 images and 14630 labels.


In [3]:
import tensorflow as tf
from tensorflow.keras import layers

def create_data_augmentation_model():
    """
    Creates a data augmentation model appropriate for animal face classification.
    """
    data_augmentation = tf.keras.Sequential([
        # Randomly flip images horizontally
        layers.RandomFlip("horizontal"),

        # Apply small random rotations (animals can appear at different angles)
        layers.RandomRotation(0.1),

        # Random brightness adjustments to account for lighting variations
        layers.RandomBrightness(factor=0.2),

        # Random zoom to simulate different distances/perspectives
        layers.RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)),

        # Slight shifts in position
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),

        # Contrast adjustments for different lighting conditions
        layers.RandomContrast(factor=0.2),
    ])

    return data_augmentation

# The augmentation can be applied as a layer in the model
# Or used during training with a data generator

In [ ]:
# Create train and validation splits (use a smaller subset for quicker testing)
max_images = min(5000, len(image_paths))  # Limit to 5000 images for faster processing
selected_paths = image_paths[:max_images]
train_paths, val_paths = train_test_split(selected_paths, test_size=0.2, random_state=42)
print(f"Training set: {len(train_paths)} images")
print(f"Validation set: {len(val_paths)} images")

# Create TensorFlow data pipelines
def create_dataset(image_paths, batch_size=32):
    # Load a small batch of images and labels for initial testing
    max_to_load = min(500, len(image_paths))  # Limit for initial testing
    paths_to_load = image_paths[:max_to_load]

    images = []
    labels = []

    print(f"Loading {len(paths_to_load)} images...")
    for i, path in enumerate(paths_to_load):
        if i % 100 == 0:
            print(f"Loaded {i}/{len(paths_to_load)} images")
        try:
            img, label = load_image_and_label(path)
            images.append(img)
            labels.append(label)
        except Exception as e:
            print(f"Error loading image {path}: {e}")

    print("Converting to numpy arrays...")
    images = np.array(images)
    labels = np.array(labels)

    print(f"Image array shape: {images.shape}")
    print(f"Label array shape: {labels.shape}")

    # Create TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset, images, labels

# Create datasets
print("Creating training dataset...")
train_dataset, train_images, train_labels = create_dataset(train_paths)
print("Creating validation dataset...")
val_dataset, val_images, val_labels = create_dataset(val_paths)

In [6]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),  # Small rotations only
    layers.RandomZoom(0.1),
    layers.RandomBrightness(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(0.1, 0.1)
])

# Visualize some augmented images
print("Visualizing augmented images...")
plt.figure(figsize=(12, 12))
for i, (image, label) in enumerate(train_dataset.take(1)):
    for j in range(9):
        augmented_image = data_augmentation(image)
        ax = plt.subplot(3, 3, j + 1)
        plt.imshow(augmented_image[0].numpy().astype("uint8"))
        plt.title(f"Confidence: {label[0][0]:.2f}")
        plt.axis("off")
plt.show()

print("Data augmentation visualization complete!")

NameError: name 'keras' is not defined

## the dataset is already split ???

Justification:

Stratification: Ensures that each fold has a similar distribution of license plate types, positions, and image conditions. This can be based on metadata like image brightness, license plate size relative to image, etc.

K-Fold: Utilizes all available data for both training and validation, which is important when working with complex models that benefit from more training examples.

Real-world Simulation: Cross-validation better simulates how the model would perform across different datasets and conditions, providing a more robust estimate of its generalization capabilities.

#Modeling

#Exceptional Work